In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}
.code_block {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-size: 75%;
    line-height: 22px; /* 5px +12px + 5px */
    #text-indent: 25px;
    #background-color: #fbfbea;
    padding: 5px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 7: K-Means clustering and filling empties
</center>
</h1>
<div class=h1_cell>
<p>
I want to turn back to the problem of filling empties. I'll focus on the `Age` column, which had 177 empties. Here is my idea: we assume that the other column values in a row with an empty age might help us determine what the age is. Maybe if the `Fare` is high then the `Age` is high?
<p>
My approach will be to see if I can group rows into clusters. Once I have these clusters, I'll go through a sub-table of all rows that have an age defined. For each row, I will find its closest cluster and add the age to that cluster. At end, I'll take the mean age in each cluster.
<p>
Now I can take a row without an age and see what cluster it falls in. When I get the right cluster, I'll use the age attached to the cluster to fill the row's age.
<p>
Whew. Kind of a complicated process. But fortunately, there is a technique in machine learning that does most of what we want. It is called K-means (jargon alert). It is a type of unsupervised learning (jargon alert). Let's load our data and then discuss.
</div>

In [2]:
import pandas as pd
import os

week = 6  # from last module

home_path =  os.path.expanduser('~')

file_path = '/Dropbox/cis399_ds1_f17/notebook_history/'

file_name = 'titanic_wrangled_w'+str(week)+'.csv'

titanic_table = pd.read_csv(home_path + file_path + file_name)

pd.__version__  # should see 0.20.3 or higher

u'0.20.3'

In [3]:
os.chdir(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')
!git pull

Already up-to-date.


In [4]:
#load the lbirary from content this week

import sys
sys.path.append(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')
from week6 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 compute_prediction	 compute_training	 f1	 find_best_splitter	 forest_builder	 
generate_table	 gig	 gini	 informedness	 k_fold	 predictor_case	 probabilities	 seeder	 tree_predictor	 
vote_taker	 


In [5]:
pd.set_option('display.max_columns', None)

<h2>
K-Means Agorithm
</h2>
<p>
<div class=h1_cell>
<p>
The general idea behind K-means is that each "row" is a list of values. You can also view each row as a multi-dimensional point. We will need to convert a Titanic row into a list of numbers. This won't be too hard. At that point, the algorithm works as follows:
<p>
The algorithm is broken into two phases. The first phase assigns rows/points to centroids. They are said to define a cluster around the centroid. The second phase recalculates the centroids from the rows/points in its cluster.
<p>
1: As a preparation step, K-Means starts by randomly selecting a number of initial rows/points to act as seeds. The number of rows/points is user determined. These seeds are called the initial centroids. There is a lot of research on picking these points other than randomly. We might look at that later.
<p>
2: Compare each row in the entire table to the centroids. The comparison uses Euclidean distance. The row is assigned to the centroid that is closest.
<p>
<b>This ends phase 1. We now have clusters of rows/points attached to the various centroids.</b> If we have 5 centroids, we have 5 lists of rows/points.
<p>
3: For phase 2, for each cluster, a new centroid is computed by taking the mean of the rows in the associated cluster. 
<p>
4: If no centroid moved following step 3 then done else repeat steps 2-3. Probably want to set a limit (e.g., 100) on the number cycles through.
</div>

<h2>
Point has 10 values
</h2>
<p>
<div class=h1_cell>
<p>
Here are the columns I'll use to cluster. We will need to turn each row (a pandas Series) into a list of 10 numbers that correspond to the 10 columns below.
</div>

In [6]:
features_used = [
 'Pclass',
 'SibSp',
 'Parch',
 'Fare',
 'emb_C',
 'emb_Q',
 'emb_S',
 'emb_nan',
 'sex_female',
 'sex_male'
]

<h2>
I'll choose 5 for k
</h2>
<p>
<div class=h1_cell>
<p>
So I want 5 centroids randomly chosen to start. I don't want duplicate centroids so replace is False. The choice of 5 is mostly arbitrary. We will see later that 4 would have worked as well as 5. But maybe 10 is much better. This is another case of needing to explore the value for hyper parameters, i.e., tuning.
</div>

In [7]:
k = 5

In [8]:
#Get the 5 starting rows and place in centroid_table

centroid_table = titanic_table.sample(n=k, replace=False, random_state=100)  # only random choice I am making
centroid_table.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_age,filled_age,emb_C,emb_Q,emb_S,emb_nan,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,ok_child,pclass_1,pclass_2,pclass_3,pclass_nan,forest_1,forest_1_type,forest_2,forest_2_type,forest_3,forest_3_type,forest_4,forest_4_type
205,0,3,"Strom, Miss. Telma Matilda",female,2.0,0,1,347054,10.4625,G6,S,0,2.0,0,0,1,0,Child,1,0,0,1,0,1,0,0,1,0,1,false_positive,1,false_positive,1,false_positive,1,false_positive
44,1,3,"Devaney, Miss. Margaret Delia",female,19.0,0,0,330958,7.8792,NaN,Q,0,19.0,0,1,0,0,Child,1,0,0,1,0,0,0,0,1,0,0,false_negative,0,false_negative,1,true_positive,1,true_positive
821,1,3,"Lulic, Mr. Nikola",male,27.0,0,0,315098,8.6625,NaN,S,0,27.0,0,0,1,0,Adult,0,1,0,0,1,0,0,0,1,0,0,false_negative,0,false_negative,0,false_negative,0,false_negative
458,1,2,"Toomey, Miss. Ellen",female,50.0,0,0,F.C.C. 13531,10.5000,NaN,S,0,50.0,0,0,1,0,Adult,0,1,0,1,0,0,0,1,0,0,0,false_negative,1,true_positive,1,true_positive,0,false_negative
795,0,2,"Otter, Mr. Richard",male,39.0,0,0,28213,13.0000,NaN,S,0,39.0,0,0,1,0,Adult,0,1,0,0,1,0,0,1,0,0,0,true_negative,0,true_negative,0,true_negative,0,true_negative


<h2>
Now to convert a row to a 10 element list of numbers
</h2>
<p>
<div class=h1_cell>
<p>
I'll take a row (as a pandas Series) and the features to use and produce a tuple of numbers. I am using tuple because we will not be changing the values of this list: an immutable list can be represented by a tuple in Python.
</div>

In [9]:
def row_to_vect(row, features):
    vect = [float(row[feature]) for feature in features]
    return tuple(vect)

In [10]:
#test it out
row_to_vect(titanic_table.iloc[0], features_used)  # the row becomes a 10-dimensional point

(3.0, 1.0, 0.0, 7.25, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0)

<h2>
Build 10 element list (tuple) for each centroid chosen previously
</h2>
<p>
<div class=h1_cell>
<p>
I am going to make a centroid a dictionary. It will contain the current centroid point of course. But it will also hold the set of points we assign it during phase 1 of K-means. I'll call that the cluster. 
</div>

In [11]:
def initialize_centroids(sample_table, features):
    k = len(sample_table)
    centroids = {}
    for i in range(k):
        row = sample_table.iloc[i]
        vector = row_to_vect(row, features)
        centroids[i] = {'centroid': vector, 'cluster': []}
        
    return centroids

In [12]:
#centroid_table was defined above as random sample of 5 rows

centroids = initialize_centroids(centroid_table, features_used)

In [13]:
for i in range(k):
    print(centroids[i])  # should be comparable to the centroid_table.head() above

{'cluster': [], 'centroid': (3.0, 0.0, 1.0, 10.4625, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0)}
{'cluster': [], 'centroid': (3.0, 0.0, 0.0, 7.8792, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0)}
{'cluster': [], 'centroid': (3.0, 0.0, 0.0, 8.6625, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0)}
{'cluster': [], 'centroid': (2.0, 0.0, 0.0, 10.5, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0)}
{'cluster': [], 'centroid': (2.0, 0.0, 0.0, 13.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0)}


<h2>
Done with set-up
</h2>
<p>
<div class=h1_cell>
<p>
We chose 5 random rows and turned them into lists of numbers (AKA a point). We store that under the key `centroid`. We also set up a `cluster` list. We will add points to this list for rows that bunch around (are closest to) the centroid.
</div>

<h2>
Phase 1 functions
</h2>
<p>
<div class=h1_cell>
<p>
We will need a function to determine the closest centroid to a row. We will use a Euclidean distance function (other distance functions are possible). Then I'll wrap it up in a phase_1 function.
</div>

In [14]:
#expects two lists of numbers, not two Series
def euclidean_distance(vect1, vect2):
    sum = 0
    for i in range(len(vect1)):
        sum += (vect1[i] - vect2[i])**2
    return sum**.5  # I claim that this square root is not needed in K-means - why not?

In [15]:
#expects row to be list of numbers, not a Series
def closest_centroid(centroids, row, k):
    min_distance = euclidean_distance(centroids[0]['centroid'], row)  # use 0th centroid as starter
    min_centroid = 0
    for i in range(1,k):
        distance = euclidean_distance(centroids[i]['centroid'], row)
        if distance < min_distance:
            min_distance = distance
            min_centroid = i
    return (min_centroid, min_distance)

In [16]:
def phase_1(centroids, table, features, k):
    for i in range(k):
        centroids[i]['cluster'] = []  # starting new phase 1 so empty out values from prior iteration
    
    #Go through every row in Titanic table (or Loan table) and place in closest centroid cluster.
    for i in range(len(table)):
        row = table.iloc[i]
        vrow = row_to_vect(row, features)
        (index, dist) = closest_centroid(centroids, vrow, k)
        centroids[index]['cluster'].append(vrow)
    
    return centroids


In [17]:
centroids = phase_1(centroids, titanic_table, features_used, k)

In [18]:
#Should be the same as above - centroids don't change in phase 1
for i in range(k):
    print(centroids[i]['centroid'])  # should be comparable to the centroid_table.head() above

(3.0, 0.0, 1.0, 10.4625, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0)
(3.0, 0.0, 0.0, 7.8792, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0)
(3.0, 0.0, 0.0, 8.6625, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0)
(2.0, 0.0, 0.0, 10.5, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0)
(2.0, 0.0, 0.0, 13.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0)


In [19]:
#Should have different cluster sizes
for i in range(k):
    print(len(centroids[i]['cluster']))

15
97
232
28
519


<h2>
Cool, now ready for phase 2
</h2>
<p>
<div class=h1_cell>
<p>
As a reminder, for each centroid, we take the mean of the points in its cluster. That will be a new point. That new point is made the new centroid.
</div>

In [20]:
#cluster is a list of points, i.e., a list of tuples.
def compute_mean(cluster):
    if len(cluster) == 0:
        return []
    the_sum = cluster[0]  # use 0th point as starter
    
    #I am using zip to pair up all points then do addition
    for i in range(1,len(cluster)):
        the_sum = [pair[0]+pair[1] for pair in zip(the_sum, cluster[i])]
    n = len(cluster)*1.0
    the_mean_point = [x/n for x in the_sum]
    return the_mean_point
    

In [21]:
#test it out
compute_mean(centroids[0]['cluster'])  # the mean point for cluster 0 which has 15 points/rows

[3.0,
 0.4,
 0.6,
 10.427773333333333,
 0.0,
 0.0,
 1.0,
 0.0,
 0.9333333333333333,
 0.06666666666666667]

<h2>
Packaging it up
</h2>
<p>
<div class=h1_cell>
<p>
I am using the phase_2 function to recompute the mean for each of the 5 clusters. I am also checking for a stopping condition: if the Euclidean distance between the old centroid and the new one is greater than a threshold (which I get to choose) *for any centroid* then can't stop.
</div>

In [22]:
def phase_2(centroids, k, threshold):
    old_centroids = []
    
    stop = True
    #Compute k new centroids and check for stopping condition
    for i in range(k):
        current_centroid = centroids[i]['centroid']
        new_centroid = compute_mean(centroids[i]['cluster'])
        centroids[i]['centroid'] = new_centroid
        if euclidean_distance(current_centroid, new_centroid) > threshold:
            stop = False  # all it takes is one

    return (stop, centroids)

In [23]:
(stop, centroids) = phase_2(centroids, k, 0.0)
print(stop)

False


<h2>
Not ready to stop
</h2>
<p>
<div class=h1_cell>
<p>
We have completed one cycle and are not ready to stop. You can see below that the centroid values have changed significantly.
</div>

In [24]:
for i in range(k):
    print(centroids[i]['centroid'])

[3.0, 0.4, 0.6, 10.427773333333333, 0.0, 0.0, 1.0, 0.0, 0.9333333333333333, 0.06666666666666667]
[2.814432989690722, 0.07216494845360824, 0.041237113402061855, 6.431832989690721, 0.061855670103092786, 0.5463917525773195, 0.3917525773195876, 0.0, 0.5360824742268041, 0.4639175257731959]
[3.0, 0.07327586206896551, 0.021551724137931036, 7.9050607758620615, 0.1336206896551724, 0.004310344827586207, 0.8620689655172413, 0.0, 0.017241379310344827, 0.9827586206896551]
[2.107142857142857, 0.07142857142857142, 0.0, 10.60714642857143, 0.10714285714285714, 0.0, 0.8928571428571429, 0.0, 0.39285714285714285, 0.6071428571428571]
[1.8959537572254335, 0.8362235067437379, 0.6204238921001927, 49.677592870905585, 0.2466281310211946, 0.04431599229287091, 0.7052023121387283, 0.0038535645472061657, 0.4489402697495183, 0.5510597302504817]


<h2>
I think we can now put it all together
</h2>
<p>
<div class=h1_cell>
<p>
We completed the first cycle by calling phase_1 and then phase_2. Let's package it up into a single function. I'll also add an upper limit on the number of cycles we want to try. I am going to package up n and the threshold as hyper parameters.
</div>

In [25]:
def k_means(table, features, k, hypers):
    n = 100 if 'n' not in hypers else hypers['n']
    threshold = 0.0 if 'threshold' not in hypers else hypers['threshold']
    
    centroid_table = table.sample(n=k, replace=False, random_state=100)  # only random choice I am making
    centroids = initialize_centroids(centroid_table, features)
    
    j = 0
    stop = False
    while( j < n and not stop):
        print('starting '+str(j+1))
        centroids = phase_1(centroids, table, features, k)
        (stop, centroids) = phase_2(centroids, k, threshold)
        j += 1
    print('done')
    return centroids

In [26]:
final_centroids = k_means(titanic_table, features_used, k, {'n':100, 'threshold':0.0})

starting 1
starting 2
starting 3
starting 4
starting 5
starting 6
starting 7
starting 8
starting 9
starting 10
starting 11
starting 12
starting 13
done


In [27]:
for i in range(k):
    print(final_centroids[i]['centroid'])

[1.8093220338983051, 0.923728813559322, 0.7457627118644068, 35.159975847457616, 0.18220338983050846, 0.0423728813559322, 0.7754237288135594, 0.0, 0.3813559322033898, 0.6186440677966102]
[1.9411764705882353, 0.0, 0.0, 0.5301470588235294, 0.058823529411764705, 0.0, 0.9411764705882353, 0.0, 0.0, 1.0]
[2.8007662835249043, 0.23180076628352492, 0.1685823754789272, 10.546654214559398, 0.13793103448275862, 0.12452107279693486, 0.7375478927203065, 0.0, 0.2950191570881226, 0.7049808429118773]
[1.1979166666666667, 1.1666666666666667, 0.5729166666666666, 96.82977187499999, 0.4166666666666667, 0.020833333333333332, 0.5416666666666666, 0.020833333333333332, 0.6041666666666666, 0.3958333333333333]
[1.0, 0.75, 1.05, 279.308545, 0.6, 0.0, 0.4, 0.0, 0.6, 0.4]


In [28]:
for i in range(k):
    print(len(final_centroids[i]['cluster']))

236
17
522
96
20


<h2>
Final results
</h2>
<p>
<div class=h1_cell>
<p>
Looks like we stopped after 13 cycles. We now have final centroids. You can also see how many rows each of the 5 centroids captured.
<p>
That is the end of the K-means process. If you want to use it on other data, you should be good to go with the functions we have above. In some cases, you won't be working with tables but instead raw values already in point form, i.e., lists of numbers. I have a project that looks at clustering sensor data using K-means. The sensor data comes already in numeric form so I can avoid the row_to_vect step.
</div>

<h2>
Ok, that's K-means but not end of problem
</h2>
<p>
<div class=h1_cell>
<p>
I now want to use my 5 centroids to predict missing values of Age, i.e., *impute* the values (jargon alert). I want to first figure out the mean age for each centroid. That is straightforward. I'll use the rows in the Titanic table that have ages. For each row with an age, I'll find its closet centroid. I'll add the `Age` value to that centroid. When I am done with all the rows, I can add up the ages for each centroid and divide by length. That will give me the mean age that goes with each centroid. I have now labeled each centroid with the mean age. That takes me from *unsupervised learning* with *non-labeled data* to *supervised learning* with *labeled data* (multiple jargon alerts).
<p>
I am now ready to create a new column `kmeans_age` in the Titanic table that is just the value of the `Age` column if that has a value. What if `Age` does not have a value? What goes in `kmeans_age` then? Glad you asked. You can now take the entire row with a NaN for `Age` and find its closest centroid. Use the mean age on that centroid to fill in your missing age value. Nice, huh. BTW: you now have a type of *Nearest centroid classifier* (jargon alert). This is very close to what we will look at in the next module, K-Nearest Neighbor (K-NN).
</div>

<h2>
You take the wheel
</h2>
<p>
<div class=h1_cell>
<p>
I am going to ask you to get me where I want to be. You have k_means. I'll ask you in this week's homework to finish the problem and produce `kmeans_age`. Well, actually, kmeans_LoanAmount.
<p>
See you over in the assignment notebook.
</div>

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

Save the table so can use it in next module.
</div>

In [ ]:
import os

week = 7  # change this each week

home_path =  os.path.expanduser('~')

file_path = ...'

file_name = 'titanic_wrangled_w'+str(week)+'.csv'

titanic_table.to_csv(home_path + file_path + file_name, index=False)